# Tutorial 5: Event-Driven Testing (Key Differentiator)

**Goal:** Validate trained policies under realistic timing constraints.

**Time:** ~15 minutes

---

## Why This Matters

You trained a policy in **synchronous mode** (Tutorial 4). But real CPS systems have:

| Real-World Factor | Training Mode | Reality |
|-------------------|---------------|---------|
| Agent tick rates | All same | Devices: 1s, Coordinators: 60s |
| Observation delay | 0ms | 50-2000ms |
| Action delay | 0ms | 100-500ms |
| Timing jitter | None | ±10-40% |

**HERON's key insight:** You can't test this by wrapping—it requires architectural support.

### Dual Execution Modes

| Mode | Use | Implementation |
|------|-----|----------------|
| **Synchronous** | Training | `env.step(actions)` — all agents step together |
| **Event-Driven** | Testing | `env.run_event_driven()` — agents tick independently |

This is built into HERON's `PettingZooParallelEnv` adapter.

## The EventScheduler

HERON's `EventScheduler` is a priority-queue based discrete event simulator:

```
EventScheduler
├── event_queue: [(t=0.0, AGENT_TICK, device_1), 
│                 (t=0.0, AGENT_TICK, device_2),
│                 (t=0.0, AGENT_TICK, coordinator),
│                 ...]
├── current_time: 0.0
└── handlers: {AGENT_TICK: tick_handler, ACTION_EFFECT: effect_handler, ...}
```

Events are processed in timestamp order. When an agent ticks, it:
1. Observes (with `obs_delay`)
2. Computes action
3. Schedules action effect (with `act_delay`)
4. Schedules next tick (with `tick_interval` + optional jitter)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from typing import Any, Dict, Optional
from dataclasses import dataclass
from gymnasium.spaces import Box

# HERON imports
from heron.scheduling import EventScheduler, Event, EventType
from heron.scheduling.tick_config import TickConfig, JitterType
from heron.core.feature import FeatureProvider
from heron.core.state import FieldAgentState
from heron.agents.field_agent import FieldAgent
from heron.agents.coordinator_agent import CoordinatorAgent
from heron.protocols.vertical import SetpointProtocol

print("Imports successful!")

## Step 1: Understanding TickConfig

Each agent has a `TickConfig` that defines its timing parameters:

In [ ]:
# Deterministic config (for training - no randomness)
training_config = TickConfig.deterministic(
    tick_interval=1.0,   # Agent ticks every 1 second
    obs_delay=0.0,       # Instant observations
    act_delay=0.0,       # Instant action effects
    msg_delay=0.0,       # Instant messages
)

print("Training config (deterministic):")
print(f"  tick_interval: {training_config.get_tick_interval():.3f}s")
print(f"  obs_delay: {training_config.get_obs_delay():.3f}s")
print(f"  act_delay: {training_config.get_act_delay():.3f}s")

# Testing config with jitter (realistic timing)
testing_config = TickConfig.with_jitter(
    tick_interval=1.0,   # Base interval
    obs_delay=0.1,       # 100ms observation delay
    act_delay=0.2,       # 200ms action delay
    msg_delay=0.05,      # 50ms message delay
    jitter_type=JitterType.GAUSSIAN,  # Gaussian noise
    jitter_ratio=0.1,    # 10% standard deviation
    seed=42,             # Reproducible randomness
)

print("\nTesting config (with 10% Gaussian jitter):")
for i in range(5):
    print(f"  Sample {i+1}: tick={testing_config.get_tick_interval():.3f}s, "
          f"obs={testing_config.get_obs_delay():.3f}s, "
          f"act={testing_config.get_act_delay():.3f}s")

## Step 2: The EventScheduler in Action

Let's see how the scheduler processes events:

In [ ]:
# Create a scheduler
scheduler = EventScheduler(start_time=0.0)

# Register agents with DIFFERENT tick intervals (heterogeneous timing)
# This is the key to realistic CPS simulation!

# Device-level agents: fast (1s)
scheduler.register_agent(
    agent_id='battery_1',
    tick_config=TickConfig.with_jitter(
        tick_interval=1.0,
        obs_delay=0.05,
        act_delay=0.1,
        jitter_type=JitterType.UNIFORM,
        jitter_ratio=0.1,
        seed=42,
    )
)

# Coordinator-level agent: slower (5s for demo, normally 60s)
scheduler.register_agent(
    agent_id='microgrid_1',
    tick_config=TickConfig.with_jitter(
        tick_interval=5.0,
        obs_delay=0.2,
        act_delay=0.5,
        jitter_type=JitterType.GAUSSIAN,
        jitter_ratio=0.05,
        seed=43,
    )
)

print(f"Scheduler: {scheduler}")
print(f"Registered agents: {list(scheduler.agent_intervals.keys())}")
print(f"Pending events: {scheduler.pending_count}")

In [ ]:
# Define a simple tick handler
tick_log = []

def tick_handler(event: Event, sched: EventScheduler):
    """Handle agent tick events."""
    tick_log.append({
        'time': event.timestamp,
        'agent': event.agent_id,
    })
    print(f"  t={event.timestamp:6.3f}s: {event.agent_id} ticks")

scheduler.set_handler(EventType.AGENT_TICK, tick_handler)

# Run simulation for 10 seconds
print("Running event-driven simulation for 10 seconds...\n")
events_processed = scheduler.run_until(t_end=10.0)

print(f"\nProcessed {events_processed} events")
print(f"Final time: {scheduler.current_time:.3f}s")

In [ ]:
# Analyze the tick patterns
import matplotlib.pyplot as plt

battery_ticks = [e['time'] for e in tick_log if e['agent'] == 'battery_1']
mg_ticks = [e['time'] for e in tick_log if e['agent'] == 'microgrid_1']

plt.figure(figsize=(12, 3))
plt.eventplot([battery_ticks, mg_ticks], lineoffsets=[1, 2], linelengths=0.8, 
              colors=['blue', 'red'], linewidths=2)
plt.yticks([1, 2], ['battery_1 (1s)', 'microgrid_1 (5s)'])
plt.xlabel('Time (s)')
plt.title('Heterogeneous Agent Tick Patterns (Event-Driven Mode)')
plt.xlim(0, 10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Battery ticks: {len(battery_ticks)} (expected ~10 for 1s interval)")
print(f"Microgrid ticks: {len(mg_ticks)} (expected ~2 for 5s interval)")

## Step 3: Using HERON's Built-in Event-Driven Support

HERON's `PettingZooParallelEnv` provides built-in methods for event-driven execution:
- `setup_event_driven()` - Creates and configures the scheduler
- `setup_default_handlers()` - Sets up standard event handlers
- `run_event_driven()` - Runs the simulation

This is the **recommended approach** - use HERON's adapter instead of raw PettingZoo:

In [ ]:
from heron.envs.adapters import PettingZooParallelEnv  # Use HERON adapter!

# Reuse features from previous tutorials
@dataclass
class BatterySOC(FeatureProvider):
    visibility = ['owner', 'upper_level']
    soc: float = 0.5
    def vector(self): return np.array([self.soc], np.float32)
    def names(self): return ['soc']
    def to_dict(self): return {'soc': self.soc}
    @classmethod
    def from_dict(cls, d): return cls(**d)
    def set_values(self, **kw):
        if 'soc' in kw: self.soc = np.clip(float(kw['soc']), 0.1, 0.9)


class SimpleBattery(FieldAgent):
    """Battery with configurable timing parameters."""
    def __init__(self, agent_id: str, upstream_id: str = None, 
                 tick_interval: float = 1.0, obs_delay: float = 0.0, act_delay: float = 0.0):
        # Store config before super().__init__
        self.tick_interval = tick_interval
        self.obs_delay = obs_delay
        self.act_delay = act_delay
        
        # Create tick_config from timing parameters
        tick_config = TickConfig.deterministic(
            tick_interval=tick_interval,
            obs_delay=obs_delay,
            act_delay=act_delay,
        )
        
        super().__init__(
            agent_id=agent_id, 
            upstream_id=upstream_id,
            config={'name': agent_id},
            tick_config=tick_config,
        )
        
        self._last_action = np.array([0.0])
    
    def set_state(self):
        """Override: Define state using correct HERON pattern."""
        self.soc_feature = BatterySOC(soc=0.5)
        self.state.features.append(self.soc_feature)
    
    def set_action(self):
        """Override: Define action space."""
        self.action.set_specs(dim_c=1, range=(np.array([-1.0]), np.array([1.0])))
    
    def observe(self, gs=None): 
        return self.state.vector()
    
    def step(self, action, dt=1.0):
        self._last_action = action
        power = float(action[0]) * 0.5
        soc = self.soc_feature.soc + power * dt / 2.0
        self.soc_feature.set_values(soc=soc)
        return {'power_mw': power, 'soc': self.soc_feature.soc}
    
    def reset(self, seed=None):
        self.soc_feature.soc = 0.5
        self._last_action = np.array([0.0])
        return self.observe()


print("Battery agent with HERON timing attributes ready!")

In [ ]:
class DualModeEnvironment(PettingZooParallelEnv):
    """Environment supporting both synchronous and event-driven modes.
    
    Using HERON's PettingZooParallelEnv gives us:
    - Built-in agent registration (register_agent)
    - Built-in event-driven setup (setup_event_driven)
    - Built-in event handlers (setup_default_handlers)
    - Built-in simulation runner (run_event_driven)
    
    This is the RECOMMENDED way to build dual-mode environments.
    """
    
    metadata = {'name': 'dual_mode_env_v0'}
    
    def __init__(self, config: Dict = None):
        # Initialize HERON's adapter (gives us HeronEnvCore features)
        super().__init__(env_id="dual_mode_env")
        
        config = config or {}
        self.max_steps = config.get('max_steps', 100)
        self.event_driven = config.get('event_driven', False)
        
        # Timing configuration
        obs_delay = 0.05 if self.event_driven else 0.0
        act_delay = 0.1 if self.event_driven else 0.0
        
        # Create and REGISTER agents with HERON
        self.agents_dict = {}
        for i in range(2):
            agent_id = f'battery_{i+1}'
            agent = SimpleBattery(
                agent_id=agent_id, 
                tick_interval=1.0,
                obs_delay=obs_delay,
                act_delay=act_delay,
            )
            self.agents_dict[agent_id] = agent
            self.register_agent(agent)  # HERON registration!
        
        # Setup PettingZoo attributes using HERON helpers
        self._set_agent_ids(list(self.agents_dict.keys()))
        self._init_spaces(
            action_spaces={aid: a.action_space for aid, a in self.agents_dict.items()},
            observation_spaces={aid: a.observation_space for aid, a in self.agents_dict.items()},
        )
        
        self._step_count = 0
        self._pending_actions = {}
        self._tick_results = {}
        
        # Setup event-driven mode using HERON's built-in support
        if self.event_driven:
            self._setup_heron_event_driven()
    
    def _setup_heron_event_driven(self):
        """Use HERON's built-in event-driven setup."""
        # This creates the scheduler and registers all agents automatically!
        self.setup_event_driven()  # HERON method from HeronEnvCore
        
        # Setup default handlers with our custom action handler
        def on_action_effect(agent_id, action):
            """Called when action effects are applied."""
            agent = self.agents_dict.get(agent_id)
            if agent:
                result = agent.step(action, dt=agent.tick_interval)
                self._tick_results[agent_id] = result
        
        self.setup_default_handlers(
            global_state_fn=lambda: {},  # No global state needed
            on_action_effect=on_action_effect,
        )
    
    @property
    def observation_space(self): return self.observation_spaces
    
    @property
    def action_space(self): return self.action_spaces
    
    def reset(self, seed=None, options=None):
        self._step_count = 0
        self._agents = self._possible_agents.copy()
        self._pending_actions.clear()
        self._tick_results.clear()
        
        # Use HERON's reset helper
        self.reset_agents(seed=seed)
        
        # Reset scheduler if in event-driven mode
        if self.event_driven and self.scheduler:
            self.scheduler.reset(start_time=0.0)
            self._setup_heron_event_driven()  # Re-register handlers
        
        obs = {aid: a.observe() for aid, a in self.agents_dict.items()}
        return obs, {aid: {} for aid in self.agents}
    
    def step(self, actions):
        self._step_count += 1
        self._timestep = self._step_count
        
        if self.event_driven:
            return self._step_event_driven(actions)
        else:
            return self._step_synchronous(actions)
    
    def _step_synchronous(self, actions):
        """Synchronous step - all agents step together."""
        results = {}
        for aid, agent in self.agents_dict.items():
            action = actions.get(aid, agent.action_space.sample())
            results[aid] = agent.step(action, dt=1.0)
        
        obs = {aid: a.observe() for aid, a in self.agents_dict.items()}
        rewards = {aid: -abs(0.5 - results[aid]['soc']) for aid in self.agents}
        
        done = self._step_count >= self.max_steps
        terminateds = {aid: done for aid in self.agents}
        terminateds['__all__'] = done
        truncateds = {aid: False for aid in self.agents}
        truncateds['__all__'] = False
        infos = {aid: results[aid] for aid in self.agents}
        
        return obs, rewards, terminateds, truncateds, infos
    
    def _step_event_driven(self, actions):
        """Event-driven step using HERON's run_event_driven()."""
        # Store actions for agents
        self._pending_actions.update(actions)
        
        # Use HERON's built-in run_event_driven
        step_duration = 1.0
        end_time = self.scheduler.current_time + step_duration
        self.run_event_driven(t_end=end_time)  # HERON method!
        
        obs = {aid: a.observe() for aid, a in self.agents_dict.items()}
        # Use soc_feature directly (correct HERON pattern)
        rewards = {aid: -abs(0.5 - self.agents_dict[aid].soc_feature.soc) 
                  for aid in self.agents}
        
        done = self._step_count >= self.max_steps
        terminateds = {aid: done for aid in self.agents}
        terminateds['__all__'] = done
        truncateds = {aid: False for aid in self.agents}
        truncateds['__all__'] = False
        infos = {aid: self._tick_results.get(aid, {}) for aid in self.agents}
        
        return obs, rewards, terminateds, truncateds, infos
    
    def render(self): pass


print("DualModeEnvironment ready (using HERON's built-in event-driven support)!")

## Step 4: Compare Synchronous vs Event-Driven Execution

In [ ]:
# Run same policy in both modes and compare
def run_episode(env, policy_fn, max_steps=50):
    """Run one episode with a given policy."""
    obs, _ = env.reset()
    total_reward = 0
    soc_history = {aid: [] for aid in env.agents}
    
    for step in range(max_steps):
        actions = {aid: policy_fn(obs[aid]) for aid in env.agents}
        obs, rewards, terminateds, _, infos = env.step(actions)
        
        total_reward += sum(rewards.values())
        for aid in env.agents:
            # Use soc_feature directly (correct HERON pattern)
            soc_history[aid].append(env.agents_dict[aid].soc_feature.soc)
        
        if terminateds.get('__all__', False):
            break
    
    return total_reward, soc_history

# Simple policy: try to maintain SOC at 0.5
def simple_policy(obs):
    soc = obs[0]
    if soc < 0.45:
        return np.array([0.5])   # Charge
    elif soc > 0.55:
        return np.array([-0.5])  # Discharge
    return np.array([0.0])       # Hold


# Test synchronous mode
sync_env = DualModeEnvironment({'max_steps': 50, 'event_driven': False})
sync_reward, sync_history = run_episode(sync_env, simple_policy)
print(f"Synchronous mode: Total reward = {sync_reward:.2f}")

# Test event-driven mode with delays
event_env = DualModeEnvironment({
    'max_steps': 50, 
    'event_driven': True,
})
event_reward, event_history = run_episode(event_env, simple_policy)
print(f"Event-driven mode: Total reward = {event_reward:.2f}")

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Synchronous
axes[0].plot(sync_history['battery_1'], 'b-', label='battery_1')
axes[0].plot(sync_history['battery_2'], 'r-', label='battery_2')
axes[0].axhline(y=0.5, color='k', linestyle='--', alpha=0.5, label='target')
axes[0].set_xlabel('Step')
axes[0].set_ylabel('SOC')
axes[0].set_title(f'Synchronous Mode (reward={sync_reward:.1f})')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Event-driven
axes[1].plot(event_history['battery_1'], 'b-', label='battery_1')
axes[1].plot(event_history['battery_2'], 'r-', label='battery_2')
axes[1].axhline(y=0.5, color='k', linestyle='--', alpha=0.5, label='target')
axes[1].set_xlabel('Step')
axes[1].set_ylabel('SOC')
axes[1].set_title(f'Event-Driven Mode (reward={event_reward:.1f})')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nPerformance gap: {(sync_reward - event_reward):.2f} reward units")
print("This gap shows the policy's sensitivity to timing variations.")

## Step 5: CPS-Calibrated Timing Configurations

For realistic testing, HERON supports timing distributions from real CPS standards:

In [ ]:
# Example: IEEE 2030-2011 SCADA timing (LogNormal distribution)
# Mean latency ~2s, with variance

# We can simulate this by setting appropriate jitter parameters
scada_timing = {
    'tick_interval': 60.0,     # SCADA polls every 60s
    'obs_delay': 2.0,          # ~2s observation latency
    'act_delay': 1.5,          # ~1.5s action latency
    'jitter_type': 'GAUSSIAN', # Approximate LogNormal with Gaussian
    'jitter_ratio': 0.4,       # 40% std (high variance for SCADA)
}

# NTCIP 1202 traffic signal timing
traffic_timing = {
    'tick_interval': 0.1,      # 100ms update rate
    'obs_delay': 0.05,         # 50ms sensor latency
    'act_delay': 0.2,          # 200ms actuator delay
    'jitter_type': 'UNIFORM',  # Uniform jitter
    'jitter_ratio': 0.2,       # +/- 20%
}

print("CPS-Calibrated Timing Configurations:")
print(f"\nSCADA (IEEE 2030): tick={scada_timing['tick_interval']}s, "
      f"obs_delay={scada_timing['obs_delay']}s")
print(f"Traffic (NTCIP 1202): tick={traffic_timing['tick_interval']}s, "
      f"obs_delay={traffic_timing['obs_delay']}s")

## Key Takeaways

### The Core HERON Contribution

**Event-driven execution cannot be achieved by wrapping PettingZoo.** It requires architectural support:

```python
# This is IMPOSSIBLE with raw PettingZoo wrappers:
# - Heterogeneous tick rates (device: 1s, coordinator: 60s)
# - Observation delays (50-2000ms)
# - Action delays (100-500ms)
# - Timing jitter (±10-40%)

# HERON provides this natively:
class MyEnv(PettingZooParallelEnv):
    def __init__(self):
        super().__init__(env_id="my_env")
        self.register_agent(agent)
        if event_driven:
            self.setup_event_driven()  # Built-in!
```

### The Testing Workflow

```
1. Train in synchronous mode (fast, deterministic)
       ↓
2. Test in event-driven mode (realistic timing)
       ↓
3. If performance degrades → retrain with timing awareness
```

### HERON Timing Configuration

| Parameter | Meaning | Example |
|-----------|---------|---------|
| `tick_interval` | How often agent acts | Device: 1s, Coord: 60s |
| `obs_delay` | Sensor latency | 50ms - 2s |
| `act_delay` | Actuator latency | 100ms - 500ms |
| `jitter_ratio` | Timing variance | 10-40% |

---

**Congratulations!** You've completed the core HERON tutorials.

**What you've learned:**
1. Agent-centric architecture (vs environment-centric)
2. Declarative visibility for observation filtering
3. Hierarchical agents with timing attributes
4. HERON adapters for RL framework compatibility
5. **Dual execution modes for realistic testing**

**Continue with advanced tutorials:**
- [06_configuration_and_datasets.ipynb](06_configuration_and_datasets.ipynb) — YAML configs, datasets
- [07_custom_protocols.ipynb](07_custom_protocols.ipynb) — Custom coordination
- [08_adding_custom_devices.ipynb](08_adding_custom_devices.ipynb) — New device types

For production: `python -m powergrid.scripts.mappo_training --test`